# Combined Map - Neighborhood Labels
Loads neighborhoods polygons and the wide labels table, then renders
a Folium map with a layer per label group (checkboxes).

In [ ]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import folium

# Config
ROOT = Path.cwd()
if not (ROOT/'data').exists():
    ROOT = ROOT.parent
if not (ROOT/'data').exists():
    raise FileNotFoundError(f"Couldn't locate 'data' directory from {Path.cwd()}")
NEI_PATH = ROOT/'data'/'neighborhoods.geojson'
OUT_DIR = Path('outputs'); OUT_DIR.mkdir(parents=True, exist_ok=True)
NW_PATH = OUT_DIR/'berlin_neighborhoods_labels_wide.csv'

# Minimal geo helpers
def ensure_wgs84(gdf):
    if gdf.crs is None: return gdf.set_crs(4326)
    return gdf.to_crs(4326) if gdf.crs.to_epsg()!=4326 else gdf

# Color maps
LABEL_COLORS = {
    'vibrancy_label': {
        '#sparse':  '#fee08b',
        '#average': '#a6d96a',
        '#vibrant': '#1a9641',
    },
    'mobility_label': {
        '#remote':         '#fee08b',
        '#moderate':       '#a6d96a',
        '#well-connected': '#1a9641',
    },
    'playgrounds_density_label': {
        '#low_playground_density':     '#56B4E9',
        '#average_playground_density': '#B9B9B9',
        '#high_playground_density':    '#0072B2',
    },
    'green_share_label': {
        '#low_green_share':     '#56B4E9',
        '#average_green_share': '#B9B9B9',
        '#high_green_share':    '#0072B2',
    },
}
def to_hashtag(label: str) -> str:
    if pd.isna(label): return '#unknown'
    s = str(label).strip().lower().replace(' ', '_').replace('-', '_')
    return s if s.startswith('#') else '#'+s
def tag_for_value(v, col):
    if pd.isna(v): return '#unknown'
    s = str(v).strip().lower()
    if col == 'playgrounds_density_label':
        return {'below average':'#low_playground_density','average':'#average_playground_density','above average':'#high_playground_density'}.get(s, to_hashtag(s))
    if col == 'green_share_label':
        return {'below average':'#low_green_share','average':'#average_green_share','above average':'#high_green_share','low_green_share':'#low_green_share','average_green_share':'#average_green_share','high_green_share':'#high_green_share'}.get(s, to_hashtag(s))
    return to_hashtag(s)
def color_for(label: str, category: str, default: str='#666666') -> str:
    return LABEL_COLORS.get(category, {}).get(label, default)

# Load polygons and wide labels
G = ensure_wgs84(gpd.read_file(NEI_PATH))
if not NW_PATH.exists():
    raise FileNotFoundError(f"Missing {NW_PATH}")
NW = pd.read_csv(NW_PATH)
NEI = G.merge(NW, on=['district','neighborhood'], how='left')
NEI = gpd.GeoDataFrame(NEI, geometry='geometry', crs=G.crs)

# Layers to render
layers = [
    'vibrancy_label',
    'mobility_label',
    'playgrounds_density_label',
    'green_share_label',
]
# Tooltip value fields per layer
value_fields = {
    'vibrancy_label': ['VV_index','venues_per_km2'],
    'mobility_label': ['connectivity_density','mobility_score'],
    'playgrounds_density_label': ['playgrounds_per_km2','n_playgrounds'],
    'green_share_label': ['green_share'],
}
aliases_map = {
    'VV_index': 'Vibrancy index',
    'venues_per_km2': 'Venues per km^2',
    'connectivity_density': 'Transit connectivity density',
    'mobility_score': 'Mobility score',
    'playgrounds_per_km2': 'Playgrounds per km^2',
    'n_playgrounds': 'Playgrounds (count)',
    'green_share': 'Green share',
}

m = folium.Map(location=[52.52, 13.405], zoom_start=10, tiles='cartodbpositron')
first = True
for col in layers:
    if col not in NEI.columns:
        continue
    fg = folium.FeatureGroup(name=col.replace('_label',''), show=first)
    def style_func(feat, col_=col):
        v = feat['properties'].get(col_)
        tag = tag_for_value(v, col_)
        return {
            'fillColor': color_for(tag, col_),
            'color': '#333333',
            'weight': 1,
            'fillOpacity': 0.7,
        }
    fields = ['district','neighborhood', col] + [f for f in value_fields.get(col, []) if f in NEI.columns]
    aliases = ['District','Neighborhood', col.replace('_',' ').title()] + [aliases_map.get(f, f) for f in fields[3:]]
    folium.GeoJson(
        NEI[[c for c in (['geometry'] + fields) if c in NEI.columns]],
        style_function=style_func,
        tooltip=folium.GeoJsonTooltip(fields=fields, aliases=aliases)
    ).add_to(fg)
    fg.add_to(m)
    first = False

folium.LayerControl(collapsed=False).add_to(m)
m.save(str(OUT_DIR/'neighborhoods_combined_map.html'))
